<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/(%ED%86%B5%ED%95%A93)_kobert_nsmc3_eda_replace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요 라이브러리 설치

In [54]:
!pip install transformers datasets accelerate scikit-learn optuna sentencepiece

In [55]:
!pip install transformers

In [56]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import torch
import optuna
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import re
import platform
import transformers
import joblib

In [57]:
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


# 80% 데이터셋

In [58]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'Colab Notebooks'  'My Drive'   nmsc3_0.5_aug_mlm.csv   이전   졸업논문


In [59]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/NSMC3_0.8_aug_eda_replace_final.csv', encoding='utf-8-sig')

In [60]:
df['label'].value_counts()

,count
label,
1,5000
0,5000


In [61]:
df_test=pd.read_excel('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_test.xlsx')

In [62]:
data_list = []
for q, label in zip(df['document'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

#from sklearn.model_selection import train_test_split
#dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(data_list)

data_list = []
for q, label in zip(df_test['document'], df_test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

test=pd.DataFrame(data_list)

train.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)

In [63]:
def clean_korean_documents(documents):
    # 텍스트 정제 (특수기호만 제거)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\sㄱ-ㅣ가-힣]', '', document)  # 특수기호만 제거, 정규 표현식
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# 한글이 아닌 문자를 제거하는 함수
def remove_non_korean(text):
    return re.sub(r'[^ㄱ-ㅣ가-힣\s]', '', text)

In [64]:
train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)


Cleaning Special Characters: 100%|██████████| 2000/2000 [00:00<00:00, 11388.03it/s]


In [65]:
# 3. 학습 데이터와 테스트 데이터에서 필요한 열 추출 및 라벨 인코딩
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

# 4. 라벨 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

In [66]:
# 5. KoBERT 모델을 위한 토크나이저 및 데이터셋 생성
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [67]:
# 텍스트를 토큰화하는 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

In [68]:
# 데이터셋 생성 및 토크나이즈
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [69]:
# PyTorch 텐서 형식으로 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT 모델 설정 및 학습 파라미터 최적화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [70]:
#CPU 측정

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU 사용량 및 속도 측정 함수
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # 시작 시간 기록
    start_cpu = psutil.cpu_percent(interval=None)  # 시작 CPU 사용량

    train_function()  # 학습 함수 호출

    end_time = time.time()  # 종료 시간 기록
    end_cpu = psutil.cpu_percent(interval=None)  # 종료 CPU 사용량

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

In [71]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 점수 계산
    return {'eval_f1': f1}  # 'eval_f1' 반환

# Optuna를 활용한 하이퍼파라미터 튜닝
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

In [72]:
training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=10,  # 에포크 수 고정
    per_device_train_batch_size=64,
    learning_rate=5.0e-05,  # 최적화된 학습률
    weight_decay=0.01,  # 최적화된 weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",  # AdamW fused 적용 (PyTorch 2.0 이상)
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # 가장 좋은 모델을 accuracy 기준으로 선택
    greater_is_better=True,  # accuracy가 높은 모델을 저장
    save_total_limit=2,  # 저장할 모델 개수 제한
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [73]:
# 최적의 파라미터로 Trainer 구성
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # 전체 학습 데이터로 평가
    compute_metrics=compute_metrics
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [74]:
# CPU 사용량 및 속도 측정
measure_cpu_usage_and_time(trainer7.train)

Epoch,Training Loss,Validation Loss,F1
1,0.293900,0.436933,0.822781
2,0.203300,0.437652,0.846956
3,0.093800,0.572009,0.824215
4,0.060600,0.497068,0.839488
5,0.052000,0.669234,0.825311
6,0.034700,0.812798,0.844382
7,0.004200,0.945992,0.838808
8,0.015600,0.920392,0.850301
9,0.002500,0.926587,0.849402
10,0.001500,0.981107,0.848651


Elapsed Time: 1090.77 seconds
Average CPU Usage: 10.55%


In [75]:
# 모델 저장
#joblib.dump(trainer7, '/content/gdrive/MyDrive/NSMC2_0.8_original.pkl')

#print("모델이 저장되었습니다.")

In [76]:
# 8. 저장된 모델로 테스트 데이터 예측 및 결과 저장
# 저장된 모델 불러오기

#loaded_model = joblib.load('/content/gdrive/MyDrive/NSMC2_0.8_original.pkl')

# 불러온 모델을 사용하여 예측 수행
predictions7 = trainer7.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

In [77]:
test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]
selected_columns7.head(10)

,text,category_prediction
0,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,폴리스스토리 시리즈는 뉴까지 버릴께 하나도 없음 최고,1
3,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
5,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,완전 감동입니다 다시봐도 감동,1
7,개들의 나오나요 나오면 보고 싶음,1
8,내 나이와 같은 영화를 지금 본 나는 감동적이다하지만 훗날 다시보면대사하나하나 그 ...,1
9,영화의 오페라화라고 해야할 작품 극단적 평갈림은 어쩔 수 없는 듯,0


In [78]:
a=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [79]:
selected_columns7['y_true']=pd.DataFrame(a)
selected_columns7 = selected_columns7.rename(columns={"category_prediction": "y_pred"})
selected_columns7.to_csv('/content/nsmc3_0.8_eda_replace_prediction.csv', encoding='utf-8-sig')

In [80]:
from sklearn.metrics import f1_score
import pandas as pd
df=pd.read_csv('/content/nsmc3_0.8_eda_replace_prediction.csv', encoding='utf-8-sig')

from sklearn.metrics import classification_report, f1_score

# y_true와 y_pred를 추출
y_true = df['y_true']
y_pred = df['y_pred']

# classification_report를 사용하여 각 클래스별 F1-score 출력
report = classification_report(y_true, y_pred, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.8781    0.8140    0.8448      1000
           1     0.8267    0.8870    0.8558      1000

    accuracy                         0.8505      2000
   macro avg     0.8524    0.8505    0.8503      2000
weighted avg     0.8524    0.8505    0.8503      2000



# 50% 데이터셋

In [81]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/NSMC3_0.5_aug_eda_replace_final.csv', encoding='utf-8-sig')

In [82]:
df_test=pd.read_excel('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_test.xlsx')

In [83]:
data_list = []
for q, label in zip(df['document'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

#from sklearn.model_selection import train_test_split
#dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(data_list)

data_list = []
for q, label in zip(df_test['document'], df_test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

test=pd.DataFrame(data_list)

train.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)

def clean_korean_documents(documents):
    # 텍스트 정제 (특수기호만 제거)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\sㄱ-ㅣ가-힣]', '', document)  # 특수기호만 제거, 정규 표현식
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# 한글이 아닌 문자를 제거하는 함수
def remove_non_korean(text):
    return re.sub(r'[^ㄱ-ㅣ가-힣\s]', '', text)


train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)

# 3. 학습 데이터와 테스트 데이터에서 필요한 열 추출 및 라벨 인코딩
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

# 4. 라벨 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

# 5. KoBERT 모델을 위한 토크나이저 및 데이터셋 생성
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 텍스트를 토큰화하는 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

# 데이터셋 생성 및 토크나이즈
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# PyTorch 텐서 형식으로 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT 모델 설정 및 학습 파라미터 최적화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)


Cleaning Special Characters: 100%|██████████| 2000/2000 [00:00<00:00, 11387.48it/s]


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [84]:
#CPU 측정

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU 사용량 및 속도 측정 함수
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # 시작 시간 기록
    start_cpu = psutil.cpu_percent(interval=None)  # 시작 CPU 사용량

    train_function()  # 학습 함수 호출

    end_time = time.time()  # 종료 시간 기록
    end_cpu = psutil.cpu_percent(interval=None)  # 종료 CPU 사용량

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

In [85]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 점수 계산
    return {'eval_f1': f1}  # 'eval_f1' 반환

# Optuna를 활용한 하이퍼파라미터 튜닝
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=10,  # 에포크 수 고정
    per_device_train_batch_size=64,
    learning_rate=5.0e-05,  # 최적화된 학습률
    weight_decay=0.01,  # 최적화된 weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",  # AdamW fused 적용 (PyTorch 2.0 이상)
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # 가장 좋은 모델을 accuracy 기준으로 선택
    greater_is_better=True,  # accuracy가 높은 모델을 저장
    save_total_limit=2,  # 저장할 모델 개수 제한
    report_to="none"
)

# 최적의 파라미터로 Trainer 구성
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # 전체 학습 데이터로 평가
    compute_metrics=compute_metrics
)

# CPU 사용량 및 속도 측정
measure_cpu_usage_and_time(trainer7.train)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,0.246700,0.494146,0.763248
2,0.190100,0.452239,0.824138
3,0.100300,0.507904,0.822673
4,0.061300,0.519863,0.843493
5,0.030600,0.690737,0.842113
6,0.019900,1.119158,0.800251
7,0.016400,1.104876,0.806209
8,0.018400,0.912477,0.835660
9,0.001900,1.053318,0.821212
10,0.003400,1.073066,0.819081


Elapsed Time: 1092.75 seconds
Average CPU Usage: 14.70%


In [86]:
# 모델 저장
#joblib.dump(trainer7, '/content/gdrive/MyDrive/NSMC2_0.5_original.pkl')

#print("모델이 저장되었습니다.")

In [87]:
# 8. 저장된 모델로 테스트 데이터 예측 및 결과 저장
# 저장된 모델 불러오기

#loaded_model = joblib.load('/content/gdrive/MyDrive/NSMC2_0.5_original.pkl')

# 불러온 모델을 사용하여 예측 수행
predictions7 = trainer7.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]

a=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

selected_columns7['y_true']=pd.DataFrame(a)
selected_columns7 = selected_columns7.rename(columns={"category_prediction": "y_pred"})
selected_columns7.to_csv('/content/nsmc3_0.5_eda_replace_prediction.csv', encoding='utf-8-sig')



In [88]:
from sklearn.metrics import f1_score
import pandas as pd

df=pd.read_csv('/content/nsmc3_0.5_eda_replace_prediction.csv', encoding='utf-8-sig')

from sklearn.metrics import classification_report, f1_score

# y_true와 y_pred를 추출
y_true = df['y_true']
y_pred = df['y_pred']

# classification_report를 사용하여 각 클래스별 F1-score 출력
report = classification_report(y_true, y_pred, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.8391    0.8500    0.8445      1000
           1     0.8480    0.8370    0.8425      1000

    accuracy                         0.8435      2000
   macro avg     0.8436    0.8435    0.8435      2000
weighted avg     0.8436    0.8435    0.8435      2000



# 10% 데이터셋

In [89]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/NSMC3_0.1_aug_eda_replace_final.csv', encoding='utf-8-sig')

df_test=pd.read_excel('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_test.xlsx')

In [90]:
data_list = []
for q, label in zip(df['document'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

#from sklearn.model_selection import train_test_split
#dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(data_list)

data_list = []
for q, label in zip(df_test['document'], df_test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

test=pd.DataFrame(data_list)

train.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)

def clean_korean_documents(documents):
    # 텍스트 정제 (특수기호만 제거)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\sㄱ-ㅣ가-힣]', '', document)  # 특수기호만 제거, 정규 표현식
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# 한글이 아닌 문자를 제거하는 함수
def remove_non_korean(text):
    return re.sub(r'[^ㄱ-ㅣ가-힣\s]', '', text)


train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)

# 3. 학습 데이터와 테스트 데이터에서 필요한 열 추출 및 라벨 인코딩
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

# 4. 라벨 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

# 5. KoBERT 모델을 위한 토크나이저 및 데이터셋 생성
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 텍스트를 토큰화하는 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

# 데이터셋 생성 및 토크나이즈
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# PyTorch 텐서 형식으로 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT 모델 설정 및 학습 파라미터 최적화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)


Cleaning Special Characters: 100%|██████████| 2000/2000 [00:00<00:00, 11546.46it/s]


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [91]:
#CPU 측정

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU 사용량 및 속도 측정 함수
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # 시작 시간 기록
    start_cpu = psutil.cpu_percent(interval=None)  # 시작 CPU 사용량

    train_function()  # 학습 함수 호출

    end_time = time.time()  # 종료 시간 기록
    end_cpu = psutil.cpu_percent(interval=None)  # 종료 CPU 사용량

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

In [92]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 점수 계산
    return {'eval_f1': f1}  # 'eval_f1' 반환

# Optuna를 활용한 하이퍼파라미터 튜닝
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=10,  # 에포크 수 고정
    per_device_train_batch_size=64,
    learning_rate=5.0e-05,  # 최적화된 학습률
    weight_decay=0.01,  # 최적화된 weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",  # AdamW fused 적용 (PyTorch 2.0 이상)
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # 가장 좋은 모델을 accuracy 기준으로 선택
    greater_is_better=True,  # accuracy가 높은 모델을 저장
    save_total_limit=2,  # 저장할 모델 개수 제한
    report_to="none"
)

# 최적의 파라미터로 Trainer 구성
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # 전체 학습 데이터로 평가
    compute_metrics=compute_metrics
)

# CPU 사용량 및 속도 측정
measure_cpu_usage_and_time(trainer7.train)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,0.120500,0.574176,0.780439
2,0.091000,0.793062,0.727422
3,0.106600,0.988920,0.628873
4,0.054800,1.642184,0.576990
5,0.015000,1.589083,0.644021
6,0.022100,1.450162,0.703341
7,0.003000,2.079739,0.639513
8,0.021000,1.806797,0.675005
9,0.002300,1.941301,0.675450
10,0.006200,1.981648,0.667196


Elapsed Time: 1087.41 seconds
Average CPU Usage: 15.60%


In [93]:
# 모델 저장
#joblib.dump(trainer7, '/content/gdrive/MyDrive/NSMC2_0.1_original.pkl')

#print("모델이 저장되었습니다.")

In [94]:
# 8. 저장된 모델로 테스트 데이터 예측 및 결과 저장
# 저장된 모델 불러오기

#loaded_model = joblib.load('/content/gdrive/MyDrive/NSMC2_0.1_original.pkl')

# 불러온 모델을 사용하여 예측 수행
predictions7 = trainer7.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]

a=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

selected_columns7['y_true']=pd.DataFrame(a)
selected_columns7 = selected_columns7.rename(columns={"category_prediction": "y_pred"})
selected_columns7.to_csv('/content/nsmc3_0.1_eda_replace_prediction.csv', encoding='utf-8-sig')



In [95]:
from sklearn.metrics import f1_score
import pandas as pd

df=pd.read_csv('/content/nsmc3_0.1_eda_replace_prediction.csv', encoding='utf-8-sig')

from sklearn.metrics import classification_report, f1_score

# y_true와 y_pred를 추출
y_true = df['y_true']
y_pred = df['y_pred']

# classification_report를 사용하여 각 클래스별 F1-score 출력
report = classification_report(y_true, y_pred, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.8269    0.7120    0.7652      1000
           1     0.7471    0.8510    0.7957      1000

    accuracy                         0.7815      2000
   macro avg     0.7870    0.7815    0.7804      2000
weighted avg     0.7870    0.7815    0.7804      2000



# 2% 데이터셋

In [96]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/NSMC3_0.02_aug_eda_replace_final.csv', encoding='utf-8-sig')

df_test=pd.read_excel('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_test.xlsx')

In [97]:
data_list = []
for q, label in zip(df['document'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

#from sklearn.model_selection import train_test_split
#dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(data_list)

data_list = []
for q, label in zip(df_test['document'], df_test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

test=pd.DataFrame(data_list)

train.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)

def clean_korean_documents(documents):
    # 텍스트 정제 (특수기호만 제거)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\sㄱ-ㅣ가-힣]', '', document)  # 특수기호만 제거, 정규 표현식
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# 한글이 아닌 문자를 제거하는 함수
def remove_non_korean(text):
    return re.sub(r'[^ㄱ-ㅣ가-힣\s]', '', text)


train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)

# 3. 학습 데이터와 테스트 데이터에서 필요한 열 추출 및 라벨 인코딩
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

# 4. 라벨 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

# 5. KoBERT 모델을 위한 토크나이저 및 데이터셋 생성
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 텍스트를 토큰화하는 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

# 데이터셋 생성 및 토크나이즈
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# PyTorch 텐서 형식으로 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT 모델 설정 및 학습 파라미터 최적화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)

#CPU 측정

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU 사용량 및 속도 측정 함수
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # 시작 시간 기록
    start_cpu = psutil.cpu_percent(interval=None)  # 시작 CPU 사용량

    train_function()  # 학습 함수 호출

    end_time = time.time()  # 종료 시간 기록
    end_cpu = psutil.cpu_percent(interval=None)  # 종료 CPU 사용량

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

Cleaning Special Characters: 100%|██████████| 2000/2000 [00:00<00:00, 12243.32it/s]


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [98]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 점수 계산
    return {'eval_f1': f1}  # 'eval_f1' 반환

# Optuna를 활용한 하이퍼파라미터 튜닝
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=10,  # 에포크 수 고정
    per_device_train_batch_size=64,
    learning_rate=5.0e-05,  # 최적화된 학습률
    weight_decay=0.01,  # 최적화된 weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",  # AdamW fused 적용 (PyTorch 2.0 이상)
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # 가장 좋은 모델을 accuracy 기준으로 선택
    greater_is_better=True,  # accuracy가 높은 모델을 저장
    save_total_limit=2,  # 저장할 모델 개수 제한
    report_to="none"
)

# 최적의 파라미터로 Trainer 구성
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # 전체 학습 데이터로 평가
    compute_metrics=compute_metrics
)

# CPU 사용량 및 속도 측정
measure_cpu_usage_and_time(trainer7.train)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,0.053700,1.494218,0.336433
2,0.044500,1.746485,0.362402
3,0.024700,2.129395,0.354104
4,0.022200,1.859862,0.433536
5,0.025700,2.385132,0.437945
6,0.000900,2.884225,0.429735
7,0.012300,3.094688,0.374219
8,0.012400,3.434867,0.365827
9,0.000400,3.383907,0.397745
10,0.011800,3.541884,0.379408


Elapsed Time: 1088.56 seconds
Average CPU Usage: 15.10%


In [99]:
# 모델 저장
#joblib.dump(trainer7, '/content/gdrive/MyDrive/NSMC2_0.02_original.pkl')

#print("모델이 저장되었습니다.")

In [100]:
# 8. 저장된 모델로 테스트 데이터 예측 및 결과 저장
# 저장된 모델 불러오기

#loaded_model = joblib.load('/content/gdrive/MyDrive/NSMC2_0.02_original.pkl')

# 불러온 모델을 사용하여 예측 수행
predictions7 = trainer7.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]

a=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

selected_columns7['y_true']=pd.DataFrame(a)
selected_columns7 = selected_columns7.rename(columns={"category_prediction": "y_pred"})
selected_columns7.to_csv('/content/nsmc3_0.02_eda_replace_prediction.csv', encoding='utf-8-sig')



In [101]:
from sklearn.metrics import f1_score
import pandas as pd

df=pd.read_csv('/content/nsmc3_0.02_eda_replace_prediction.csv', encoding='utf-8-sig')

from sklearn.metrics import classification_report, f1_score

# y_true와 y_pred를 추출
y_true = df['y_true']
y_pred = df['y_pred']

# classification_report를 사용하여 각 클래스별 F1-score 출력
report = classification_report(y_true, y_pred, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.9810    0.1030    0.1864      1000
           1     0.5266    0.9980    0.6895      1000

    accuracy                         0.5505      2000
   macro avg     0.7538    0.5505    0.4379      2000
weighted avg     0.7538    0.5505    0.4379      2000

